In [1]:
from googlesearch import search
import Algorithmia
import abc

In [2]:
class PredictAlgorithm(abc.ABC):
    @abc.abstractmethod
    def Response(self, algo=None, input = '', specifyResponse=''):
        pass
    
class AlgorithmiaUtils(PredictAlgorithm):
    def __init__(self, input=[], optionAlgorithimia='', reqtimeout=300, client = 'sim8t0NCJjHfpgWiLSuOfnlhrPe1', specifyResponse=''):
        self.input = input
        self.optionAlgorithimia = optionAlgorithimia
        self.reqtimeout = reqtimeout
        self.client = client
        self.specifyResponse = specifyResponse
        
    def CreateAlgo(self):
        client = Algorithmia.client(self.client)
        algo = client.algo(self.optionAlgorithimia)
        algo.set_options(timeout=self.reqtimeout)
        
        return algo
    
    def Response(self, algo=None, input = '', specifyResponse=''):
        
        if(algo==None):
            algo = self.CreateAlgo()
        if(input==''):
            input = self.input
        if(specifyResponse==''):
            specifyResponse = self.specifyResponse
        
        response = algo.pipe(input).result
        
        if(specifyResponse!=''):
            return response[specifyResponse]
        
        return response

In [18]:
class AnswareQuestions(abc.ABC):
    @abc.abstractmethod
    def AnswareTheQuestion(self, question='', seq_lenth=512, isdebug=False, language='', isjuridic=None):
        pass

class AnswareQuestionsByGoogleSearch(AnswareQuestions):
    def __init__(self, question='', language='pt', domainLevel = 'com', dList = [], stopSearch=1, juridic=False, isdebug=False):
        self.question = question
        self.language = language
        self.domainLevel = domainLevel
        self.dList = dList
        self.stopSearch = stopSearch
        self.juridic = juridic
        self.isdebug = isdebug
        
    def SearchResponseLinksOfQuestionOnGoogle(self):
        listDomain = self.dList
        
        if(self.juridic and listDomain==[]):
            listDomain = ['jus.com.br', 'www.procon.pr.gov.br', 'jusbrasil.com.br']
        
        links = list(search(self.question, lang=self.language, tld=self.domainLevel, domains= listDomain, stop=self.stopSearch))
        
        if(self.isdebug):
            print("Step 1 - Founded Links:")
            print(links)
            print('\r\n')
        
        return links
    
    def AlgorithmGetTextFromLink(self):
        text = []
        algorithmia = AlgorithmiaUtils(optionAlgorithimia='web/AnalyzeURL/0.2.17')
        
        responseList = self.SearchResponseLinksOfQuestionOnGoogle()
        
        for link in responseList:
            response = algorithmia.Response(input = [link, self.question], specifyResponse='text')
            text.append(response)

        if(self.isdebug):
            print("Step 2 - Text from links:")
            print(text)
            print('\r\n')
            
        return text
    
    def AlgorithmPredict(self, text):
        searchinput = {
          "passage": text,
          "question": self.question
        }

        algorithmia = AlgorithmiaUtils(optionAlgorithimia='allenai/machine_comprehension/0.1.2', input=searchinput)

        return algorithmia.Response()
    
    def PredictByGoogleSearchAnsware(self, contexts, seq_len=512, debug=False):
        split_context = []
        
        for context in contexts:
            for i in range(0, len(context), seq_len):
                split_context.append(context[i:i+seq_len])

        split_context = contexts

        predicts = []

        for x in split_context:
            p = self.AlgorithmPredict(x)

            if(self.isdebug):
                print("Step 3 - Answare AlgorithmPredict:")
                print(p)
                print('\r\n')
                
            predicts.append(p['best_span_str'])

        if predicts: return predicts

        return 'No answer'
    
    def AnswareTheQuestion(self, question='', seq_lenth=512, isdebug=None, language='', isjuridic=None):
        if(question!=''):
            self.question = question
        
        if(language!=''):
            self.language = language
            
        if(isjuridic!=None):
            self.juridic
        
        if(isdebug!=None):
            self.isdebug = isdebug
        
        if(self.isdebug):
            print('Debug Option Activate')
            print('\r\n')
        
        answare = self.AlgorithmGetTextFromLink()
        prediction = self.PredictByGoogleSearchAnsware(contexts=answare, seq_len=seq_lenth, debug=isdebug)
        
        return prediction

In [19]:
aw = AnswareQuestionsByGoogleSearch(isdebug=True)

rep = aw.AnswareTheQuestion(question='What color is the sky', language='en')
rep

Debug Option Activate


Step 1 - Founded Links:
['https://www.universetoday.com/74020/what-color-is-the-sky/']


Step 2 - Text from links:
['Universe Today Space and astronomy news [/caption] If you are a parent or are old enough to babysit younger relatives there is one question children ask that stumps most adults. It’s what color is the sky or why is the sky blue. This article will tell you why and do it in as simple a way as possible so that the next time a kids ask the question you have a good answer. To understand why the sky is blue you need to remember how color works. Color is largely caused by how well an object absorbs the light spectrum. When you see a blue sky you only see blue because all the other colors were absorbed in the air. Any object with color works that way. For example a red ball is read because all the colors of light are absorbed by the ball except for red. This reflected light is what gives the object color. This is what happens with the sky. The atmosphere 

['blue']